# MM compare log analysis
Set `run_id` to the directory under `log/` you want to inspect (e.g., `test_seed1`, `test_seed2`).

Loads the summary log, fundamental path, order book (if available), and samples of agent logs for the single market maker, ZI, HBL, value, and momentum cohorts.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)

# Change this to point to a different log directory (relative to the repo root)
run_id = 'test_seed1'
log_dir = Path('..') / 'log' / run_id
print('Log dir:', log_dir.resolve())
sorted(log_dir.iterdir())[:10]

## Summary log

In [ ]:
summary = pd.read_pickle(log_dir / 'summary_log.bz2')
wide_summary = (
    summary
    .pivot_table(index=['AgentID', 'AgentStrategy'], columns='EventType', values='Event', aggfunc='first')
)
wide_summary['PNL'] = wide_summary['ENDING_CASH'] - wide_summary['STARTING_CASH']
wide_summary.reset_index().head(20)

In [ ]:
# Aggregate PnL by agent strategy
strategy_pnl = wide_summary.groupby(level=1)['PNL'].sum().sort_values(ascending=False)
strategy_pnl.to_frame('PNL')

In [ ]:
# Top/bottom PnL by individual agent
wide_summary.reset_index().sort_values('PNL', ascending=False).head(10)

In [ ]:
# Market maker PnL slice
mm_summary = (
    wide_summary.reset_index()
    .query("AgentStrategy.str.contains('MarketMaker')", engine='python')
    .sort_values('PNL', ascending=False)
)
mm_summary[['AgentID', 'AgentStrategy', 'PNL']].head(10)

## Fundamental path

In [ ]:
# Derive ticker from ORDERBOOK filename (e.g., ORDERBOOK_AAPL_FULL.bz2)
orderbook_file = next(log_dir.glob('ORDERBOOK_*_FULL.bz2'))
ticker = orderbook_file.name.split('_')[1]

fundamental_path = log_dir / f'fundamental_{ticker}.bz2'
fundamental = pd.read_pickle(fundamental_path).reset_index()

fundamental.plot(x='FundamentalTime', y='FundamentalValue', legend=False, figsize=(10, 4))
plt.ylabel('Fundamental value')
plt.title(f'Fundamental path for {ticker}')
plt.tight_layout()

## Order book (top of book)

If the order book log includes best bid/ask price columns, the cell below will extract and plot mid/spread. Otherwise it shows the available columns so you can adapt.

In [ ]:
orderbook = pd.read_pickle(orderbook_file)
print('Orderbook columns:', list(orderbook.columns))

# Heuristic search for best bid/ask price columns
def find_col(columns, substrings):
    for col in columns:
        name = ' '.join(col) if isinstance(col, tuple) else str(col)
        lname = name.lower()
        if all(sub in lname for sub in substrings):
            return col
    return None

bid_price_col = find_col(orderbook.columns, ['best', 'bid']) or find_col(orderbook.columns, ['bid', 'price'])
ask_price_col = find_col(orderbook.columns, ['best', 'ask']) or find_col(orderbook.columns, ['ask', 'price'])

if bid_price_col and ask_price_col:
    top = orderbook[[bid_price_col, ask_price_col]].copy()
    top.columns = ['best_bid_price', 'best_ask_price']
    top['mid'] = (top['best_bid_price'] + top['best_ask_price']) / 2
    top['spread'] = top['best_ask_price'] - top['best_bid_price']
    ax = top[['mid']].plot(figsize=(10, 4), title='Mid price')
    plt.tight_layout()
    ax = top[['spread']].plot(figsize=(10, 3), title='Quoted spread')
    plt.tight_layout()
else:
    display(orderbook.head())

## Agent logs

Helper to load any agent log, then example slices for a market maker, ZI, HBL, value, and momentum agent. Adjust filenames as needed.

In [ ]:
def load_agent_log(filename):
    return pd.read_pickle(log_dir / filename)

all_agents = sorted(
    f.name for f in log_dir.glob('*.bz2')
    if 'summary' not in f.name and 'ORDERBOOK' not in f.name and 'fundamental' not in f.name
)
print('Total agent logs:', len(all_agents))
all_agents[:10]

In [ ]:
# Market maker example (single MM)
mm_files = [f for f in all_agents if 'MARKET_MAKER' in f]
if mm_files:
    mm_log = load_agent_log(mm_files[0])
    display(mm_log.head())
    print('Columns:', list(mm_log.columns))
else:
    print('No market maker logs found')

In [ ]:
# Zero Intelligence agent example
zi_files = [f for f in all_agents if 'ZI_AGENT' in f]
if zi_files:
    zi_log = load_agent_log(zi_files[0])
    display(zi_log.head())
else:
    print('No ZI agent logs found')

In [ ]:
# Heuristic Belief Learning agent example
hbl_files = [f for f in all_agents if 'HBL_AGENT' in f]
if hbl_files:
    hbl_log = load_agent_log(hbl_files[0])
    display(hbl_log.head())
else:
    print('No HBL agent logs found')

In [ ]:
# Value agent example
value_files = [f for f in all_agents if 'ValueAgent' in f]
if value_files:
    value_log = load_agent_log(value_files[0])
    display(value_log.head())
else:
    print('No value agent logs found')

In [ ]:
# Momentum agent example
momentum_files = [f for f in all_agents if 'MOMENTUM_AGENT' in f]
if momentum_files:
    momentum_log = load_agent_log(momentum_files[0])
    display(momentum_log.head())
else:
    print('No momentum agent logs found')

## Optional: Mark-to-market trajectory for the market maker

If the MM log contains `Holdings`, `Cash`, and a price column (`MidPrice`/`LastPrice`), the helper below will compute and plot running mark-to-market PnL.

In [ ]:
def plot_mm_mark_to_market(mm_df):
    colmap = {c.lower(): c for c in mm_df.columns}
    cash_col = colmap.get('cash')
    holdings_col = colmap.get('holdings') or colmap.get('inventory')
    price_col = colmap.get('midprice') or colmap.get('mid_price') or colmap.get('lastprice') or colmap.get('last_price')
    time_col = colmap.get('time') or colmap.get('timestamp')

    needed = [cash_col, holdings_col, price_col, time_col]
    if any(c is None for c in needed):
        print('Missing columns for mark-to-market:', needed)
        return

    pnl = mm_df[cash_col] + mm_df[holdings_col] * mm_df[price_col]
    plot_df = pd.DataFrame({
        'time': mm_df[time_col],
        'mark_to_market': pnl
    }).set_index('time')
    plot_df.plot(title='Market maker mark-to-market', figsize=(10, 4))
    plt.tight_layout()

# Example usage (uncomment after confirming columns above)
# if mm_files:
#     plot_mm_mark_to_market(mm_log)